In [1]:
import utils
import numpy as np

from tqdm import tqdm
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [2]:
# Note: If this cell takes too long to complete, re-run.
dji = utils.get_dji()
spy = utils.get_spy()
qqq = utils.get_qqq()

symbols = set(dji['Symbol']) | set(spy['Symbol']) | set(qqq['Symbol'])

DJI: Contains 30 tickers.
SPY: Contains 503 tickers.
QQQ: Contains 65 tickers.


In [3]:
args = utils.load_args()
startdate, enddate = utils.get_dates(args)

market_return, riskfree_return, nonholiday_dates = utils.get_benchmark_data(args.benchmark.market, args.benchmark.riskfree, startdate, enddate)
market_data = utils.get_pdr_data(args.benchmark.market, startdate-utils.parse_period_date(args.sharpe.period), enddate, progress=False, clean=False)
stocks_data = utils.get_pdr_data(symbols, startdate-utils.parse_period_date(args.sharpe.period), enddate, progress=True, clean=True)

[*********************100%%**********************]  516 of 516 completed

12 Failed downloads:
['GEHC', 'CEG', 'KVUE', 'VLTO', 'ABNB']: Exception("%ticker%: Data doesn't exist for startDate = 1493611200, endDate = 1604203200")
['CERN', 'BRK.B', 'FISV', 'CTXS', 'BBBY', 'ATVI']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2017-05-01 00:00:00 -> 2020-11-01 00:00:00)')


In [4]:
sharpe_update_dates = utils.linspace_datetime(nonholiday_dates, startdate, enddate, delta=utils.parse_period_date(args.sharpe.period))
rebalance_update_dates = utils.linspace_datetime(nonholiday_dates, startdate, enddate, delta=utils.parse_period_date(args.rebalance.period))

In [5]:
portfolio = utils.Portfolio(args.display.initial_balance)
stocks_return = utils.compute_return(stocks_data, was_annual=False, retain_symbols=True)

for dt in tqdm(nonholiday_dates, desc="nonholiday_dates"):
    if dt in rebalance_update_dates:
        # Compute sharpe.
        startdate_temp = dt - utils.parse_period_date(args.sharpe.period)
        enddate_temp = dt
        sharpe_df = utils.get_stocks_sharpe_from_data(stocks_data, args.benchmark.riskfree, startdate_temp, enddate_temp)
        
        # Rebalance portfolio.
        market_sharpe = utils.get_stocks_sharpe_from_data(market_data, args.benchmark.riskfree, startdate_temp, enddate_temp)
        portfolio.rebalance(sharpe_df.head(args.params.no_of_companies), min_threshold=market_sharpe[0])

        if args.display.verbose:
            print("Date:", dt)
            print("Market Sharpe:", market_sharpe[0])
            portfolio.display()
            print('-'*80)
    
    # Update portfolio with today's returns.
    portfolio.update(stocks_return, dt)

nonholiday_dates:  11%|█         | 83/754 [00:00<01:09,  9.71it/s]

Date: 2017-11-01 00:00:00
Market Sharpe: 0.12884898868195624
        !Utility   %Weight     $Value
Ticker                               
CBOE    0.316170  0.397801  39.780111
PYPL    0.245000  0.308256  30.825559
MTD     0.233625  0.293943  29.394330
--------------------------------------------------------------------------------


nonholiday_dates:  39%|███▉      | 297/754 [00:00<00:16, 27.14it/s]

Date: 2018-07-11 00:00:00
Market Sharpe: 0.006128572794035204
        !Utility   %Weight     $Value
Ticker                               
DXCM    0.209748  0.335357  32.831812
NFLX    0.208545  0.333434  32.643506
PTC     0.207153  0.331209  32.425644
--------------------------------------------------------------------------------


nonholiday_dates:  60%|█████▉    | 449/754 [00:00<00:05, 52.12it/s]

Date: 2019-03-21 00:00:00
Market Sharpe: -0.025754631597981952
        !Utility   %Weight     $Value
Ticker                               
ENPH    0.177302  0.362297  38.632976
AMT     0.165804  0.338802  36.127581
BALL    0.146277  0.298901  31.872767
--------------------------------------------------------------------------------


nonholiday_dates:  82%|████████▏ | 622/754 [00:01<00:01, 97.48it/s]

Date: 2019-12-02 00:00:00
Market Sharpe: 0.1269537705281962
        !Utility   %Weight     $Value
Ticker                               
GNRC    0.236562  0.337830  55.884162
AAPL    0.235713  0.336617  55.683521
BLDR    0.227965  0.325553  53.853282
--------------------------------------------------------------------------------


nonholiday_dates: 100%|██████████| 754/754 [00:01<00:00, 487.12it/s]

Date: 2020-08-11 00:00:00
Market Sharpe: 0.01439557541156878
        !Utility   %Weight     $Value
Ticker                               
EQT     0.176561  0.356188  87.152813
ETSY    0.175062  0.353164  86.412752
WST     0.144073  0.290648  71.116239
--------------------------------------------------------------------------------


In [6]:
# Final portfolio.
portfolio.display()

        !Utility   %Weight     $Value
Ticker                               
EQT     0.176561  0.343262  81.299670
ETSY    0.175062  0.347938  82.407264
WST     0.144073  0.308801  73.137768


In [7]:
print("Dates:           ", nonholiday_dates[0].date(), "to", nonholiday_dates[-1].date())
print("Portfolio return:", 100*(portfolio.value-args.display.initial_balance)/args.display.initial_balance, "%")

Dates:            2017-11-01 to 2020-10-29
Portfolio return: 136.84470282830313 %


In [8]:
test_data = utils.get_pdr_data(['VTI'], startdate, enddate, progress=False, clean=False)
test_return = utils.compute_return(test_data, was_annual=False, retain_symbols=True)

print("Test return:", 100*(test_data.tail(1).values[0] - test_data.head(1).values[0]) / test_data.head(1).values[0] , "%")

Test return: 26.087286631577445 %


In [9]:
# Couple of hings to try:
# - Double check performance when rebalancing.
# - Try Sortino's ratio instead.
# - Try also using Market Cap.